<a href="https://colab.research.google.com/github/RomaricKanyamibwa/Normalisation-de-concepts-medicaux/blob/master/CNN_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt
import logging


Using TensorFlow backend.


In [0]:
!pip3 install --upgrade gensim

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
from gensim.models import Word2Vec
import gensim
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np

In [0]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/Word2VecModels/
!ls -l drive/Coolab/

In [99]:
google_drive="drive/Coolab/"
df_disorders=pd.read_csv("drive/Coolab/Disorders.csv",names=["Code","Disorder","GenDisorder"],sep="\t") 
num_rows=df_disorders.shape[0]

#step 2:Clean data
count_empty=df_disorders.isnull().sum()
count_non_empty=count_empty[count_empty==0]
print(count_non_empty)
#remove empty rows
df_disorders=df_disorders[count_non_empty.keys()]

Code           0
Disorder       0
GenDisorder    0
dtype: int64


In [100]:
Codes=df_disorders['Code']#Y vector (outputs)
Disorders=df_disorders['Disorder']#X vector(test)
GenDisorders=df_disorders['GenDisorder']#X vector (train)
len(Disorders)
len(Codes)

623509

In [0]:
sample_size=10000#Disorders.size//10
#train
Disorders=Disorders.sample(sample_size,random_state=10)
Codes=Codes[Disorders.index]


In [0]:
#test set
testDisorders=df_disorders['Disorder']
testDisorders=testDisorders.drop(Disorders.index)
testDisorders=Disorders.sample(sample_size//5,random_state=10)
testCodes=Codes[testDisorders.index]

In [103]:
print(Disorders.index)

Int64Index([222160,  56824, 426769, 154154, 173255, 125527, 183515, 548570,
            552073, 249167,
            ...
            445469,  46474,  25399, 178495, 251717, 166369,  39648,  83826,
            373824,  36545],
           dtype='int64', length=10000)


In [104]:
for i,j in enumerate(zip(Codes,Disorders)):
  print(j)
  if(i>10):break

('C0275585', 'morax-axenfeld; conjunctivitis')
('C1392692', 'malposition; congenital, clavicle')
('C2118738', 'self-inflicted overdose of anterior pituitary hormones')
('C0752150', 'headache syndrome, hypnic')
('C0343713', 'gonococcal septicaemia affecting skin')
('C1385072', 'disease (or disorder); pulmonary, vessels')
('C0010414', 'cryptococcus neoformans; infection')
('C0013772', 'abuse aged')
('C2874939', 'other natural environment type phobia')
('C0839928', 'acute haematogenous osteomyelitis, multiple sites')
('C2025970', 'cellulitis of right thumb')
('C2892793', 'driver of three-wheeled motor vehicle injured in collision with car, pick-up truck or van in nontraffic accident, sequela')


In [105]:
for i,j in enumerate(zip(testCodes,testDisorders)):
  print(j)
  if(i>10):break

('C0570064', 'broad spectrum penicillins adverse reaction')
('C2187152', 'retinal detachment of both eyes without retinal defect')
('C0345375', 'short; femur')
('C4024679', 'overgrowth and curving of toenails')
('C4025395', 'comma-shaped scaphoid')
('C1410696', 'syphilis; paraplegia')
('C0005604', 'birth trauma nos')
('C0031190', 'familial persistent pulmonary hypertension of the newborn')
('C2859542', 'unsp inj blood vessels at hip and thi lev, right leg, subs')
('C0018995', 'hematochromatosis')
('C0156387', 'hymen; rigid')
('C0008512', 'unspecified chorioretinal scars')


In [0]:
def removed_elements_generator(input_list):
    """This function creates an id generator of the empty tokenized elements"""
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line_process=remove_stopwords(line)
        line_process=gensim.utils.simple_preprocess(line_process,min_len=3,max_len=30)
        if not line_process:
          print(line_process,"before preprocess-",i,":",line)
          RemovedElements.append(i)
          #line_process=line.split()
          yield i

def input_tokenize(input_list,RemovedElements):
    """This creates a generator of the tokenized elements """
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line_process=remove_stopwords(line)
        line_process=gensim.utils.simple_preprocess(line_process,min_len=3,max_len=30)
        if not line_process:
          print(line_process,"before preprocess-",i,":",line)
          RemovedElements.append(i)
          #line_process=line.split()
        else:    
          yield line_process

In [0]:
logging.info ("Tokenization")
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
#GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("End of Tokenization")

In [108]:
for i,s in enumerate(Disorders_corpus):
  #print(s)
  k=1
  #if i >:break
    
print(RemovedElements)

[] before preprocess- 469 : ad18
[] before preprocess- 767 : hs5
[] before preprocess- 1235 : t(17;22)(q22;q13)
[] before preprocess- 1238 : rp69
[] before preprocess- 2640 : t(17;21)(q11;q22)
[] before preprocess- 3070 : ts
[] before preprocess- 4897 : ml i
[] before preprocess- 4960 : rp15
[] before preprocess- 6175 : cd332 c.1645a>c
[] before preprocess- 7334 : cd332 c.1144t>c
[] before preprocess- 8473 : ad5
[] before preprocess- 8543 : t(2;17)(p23;q35)
[] before preprocess- 8678 : t2d
[] before preprocess- 9224 : k20r
[469, 767, 1235, 1238, 2640, 3070, 4897, 4960, 6175, 7334, 8473, 8543, 8678, 9224]


In [0]:
model_dir=google_drive+"Disorders"
model=Word2Vec.load(model_dir)


In [0]:
def getsentence_matrix(sentence,gensim_model,nrows=30):
  #print("sent:",sentence)
  sentence_matrix=[gensim_model.wv.get_vector(token) for token in sentence]
 
  sentence_matrix=np.matrix(sentence_matrix)
#  print("Before padding:")
#  pprint(sentence_matrix.T[:2,:])
  #print("size before padding:",nrows-sentence_matrix.shape[0])
  if(nrows-sentence_matrix.shape[0]<0 or sentence_matrix.shape[1]<=0):
    print("Sentence:",sentence)
    print("MatrixShape:",sentence_matrix.shape)
    print(sentence_matrix)
  sentence_matrix=np.lib.pad(sentence_matrix,((0,max(0,nrows-sentence_matrix.shape[0])),(0,0)), 'constant', constant_values=(0))
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
#  print("After padding:")
#  pprint(sentence_matrix.T[:2,:])
  #print("shape=",sentence_matrix.shape)
  return sentence_matrix#.T

In [0]:
def get_termes_matrices(Corpus,model):
  
#  pprint(Corpus)
  matrix_list=[]
  IndexOfEmptyElem=[]
  nrows=30
  empty=True
  for i,sentence in enumerate(Corpus):
    if not(not sentence):
      if empty==True:
        matrix_list=np.array([getsentence_matrix(sentence,model,nrows=30)])
        empty=False
        #print(matrix_list[0].shape)
      else:
        #print(i,":",getsentence_matrix(sentence,model).shape)
        matrix_list=np.append(matrix_list,[getsentence_matrix(sentence,model,nrows=30)],axis=0)
      #print("i:",i)#,matrix_list)
      if(i%1000==0):
        print("i:",i)
       # break
    else:
      print("Empty element's index:",i)
      IndexOfEmptyElem.append(i)
      #print(np.zeros((5,5)))
      matrix_list=np.append(matrix_list,[np.zeros((nrows,model.wv.vector_size))],axis=0)
  print("length:",len(matrix_list))
  print("shape=",matrix_list.shape)
  #print(matrix_list)
  return matrix_list,IndexOfEmptyElem


In [0]:
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
testRemovedElements=[]
testDisorders_corpus=(input_tokenize(testDisorders,testRemovedElements))

In [0]:
RemovedElementsIter=removed_elements_generator(Disorders)
testRemovedElementsIter=removed_elements_generator(testDisorders)

In [113]:
print(list(RemovedElementsIter))
print("Test")
print(list(testRemovedElementsIter))

[] before preprocess- 469 : ad18
[] before preprocess- 767 : hs5
[] before preprocess- 1235 : t(17;22)(q22;q13)
[] before preprocess- 1238 : rp69
[] before preprocess- 2640 : t(17;21)(q11;q22)
[] before preprocess- 3070 : ts
[] before preprocess- 4897 : ml i
[] before preprocess- 4960 : rp15
[] before preprocess- 6175 : cd332 c.1645a>c
[] before preprocess- 7334 : cd332 c.1144t>c
[] before preprocess- 8473 : ad5
[] before preprocess- 8543 : t(2;17)(p23;q35)
[] before preprocess- 8678 : t2d
[] before preprocess- 9224 : k20r
[469, 767, 1235, 1238, 2640, 3070, 4897, 4960, 6175, 7334, 8473, 8543, 8678, 9224]
Test
[] before preprocess- 1568 : t2d
[1568]


In [0]:
listNull=[457,469,767]
for i,k in enumerate(zip(Disorders_corpus,Disorders)):
  if(i in listNull):
    print(k)
    line=remove_stopwords(k[1])
    print("#remove_stopwords:",line)
    line=gensim.utils.simple_preprocess(line,min_len=1)
    print("#simple_preprocess:",line)

In [0]:
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
testRemovedElements=[]
testDisorders_corpus=(input_tokenize(testDisorders,testRemovedElements))


In [0]:
MatrixList,empty=get_termes_matrices(Disorders_corpus,model)

In [38]:
print("test sample")
testMatrixList,testempty=get_termes_matrices(testDisorders_corpus,model)

test sample
i: 0
[] before preprocess- 324 : ea4
i: 1000
length: 1999
shape= (1999, 30, 300)


In [0]:
def remove_element_byIndex(panda_dataset,listOfIndexes):
  Indexes=[]
  for elem in listOfIndexes:
    Indexes.append(panda_dataset[elem:elem+1].index[0])
  
  print(Indexes)
  return panda_dataset.drop(Indexes)

In [98]:


print(Codes.reindex([224267,110224]))
#print(Disorders[RemovedElements[0]:RemovedElements[0]+1])
print(Codes[RemovedElements[0]:RemovedElements[0]+1])
#print(Disorders[RemovedElements[1]:RemovedElements[1]+1])
#po=pd.Int64Index([], dtype='int64')
#po.append(224267)

print("Before drop:",len(Codes))
#Codes=remove_element_byIndex(Codes,RemovedElements)
print("After drop:",len(Codes))
print(testempty)
print("after drop")

224267    NaN
110224    NaN
Name: Code, dtype: object
525893    C0161721
Name: Code, dtype: object
Before drop: 10000
After drop: 10000
[]
after drop


In [0]:
def get_code_classes(Codes,gensim_model):
  unique_Codes=list(set(Codes))
  nb_classes=len(unique_Codes)
  print("number of codes:",nb_classes)
  Code_vectors=[gensim_model.wv.get_vector(token) for token in Codes]
  Code_dict=dict(zip(Codes,Code_vectors))
  #inv_Code_dict=dict(zip(Code_vectors,Codes))
  #print(Code_vectors[0][0:5])
  #print("'C0009021'",Code_dict['C0009021'][0:5])
  return nb_classes,Code_vectors,Code_dict

In [0]:
nb_classes,Code_vectors,Code_dict=get_code_classes(Codes,model)

In [0]:
Codes[222160]
print("Dict:",Code_dict["C0275585"][0:5])
print("Vect:",Code_vectors[0][0:5])

Dict: [-0.20602292 -0.07620893  0.05917473  0.12279531  0.05850269]
Vect: [-0.20602292 -0.07620893  0.05917473  0.12279531  0.05850269]


In [0]:
testnb_classes,testCode_vectors,testCode_dict=get_code_classes(testCodes,model)

testCodes[244939]
print("Dict:",testCode_dict["C0341679"][0:5])
print("Vect:",testCode_vectors[0][0:5])

number of codes: 1988
Dict: [ 0.10499874  0.155534    0.00348204 -0.0099629   0.04916005]
Vect: [ 0.10499874  0.155534    0.00348204 -0.0099629   0.04916005]


In [0]:
batch_size = 128
num_classes = nb_classes
epochs = 10

# input dimensions
size_x, size_y = MatrixList[0].shape
keras_model = Sequential()
print(type(np.array(MatrixList)))

# load the MNIST data set, which already splits into train and test sets for us
x_train, y_train = np.array(MatrixList),np.array(Code_vectors)
x_test, y_test = np.array(testMatrixList),np.array(testCode_vectors)
print(type(x_train[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [0]:
print((x_train.shape))
print((x_test.shape))

(20000,)
(2000,)


In [0]:
# reshape the data into a 4D tensor - (sample_number, x_size, y_size, num_channels)
# we only have a single channel
x_train = x_train.reshape(x_train.shape[0], size_x, size_y, 1)
x_test = x_test.reshape(x_test.shape[0], size_x, size_y, 1)
input_shape = (size_x, size_y, 1)

ValueError: ignored